### Для входа нужны
- users.csv
- data_mos.xlsx

### Выход:
- _users_features.csv

In [1]:
import numpy as np
import pandas as pd

In [2]:
df_users = pd.read_csv('users.csv')

In [3]:
df_users=df_users.rename(columns={'уникальный номер':'user_id','пол':'sex','дата рождения':'birthday','адрес проживания':'address'})


### Добавляем возраст

In [4]:
df_users["age"] = (pd.to_datetime("today") - pd.to_datetime(df_users["birthday"]))
df_users['age'] = (df_users['age'].dt.days / 365).astype(int)

### Добавляем пол

In [5]:
df_users['sex'].replace(('Женщина', 'Мужчина'), ('Female', 'Male'), inplace=True)

In [6]:
df_users.drop(columns=['дата создание личного дела','birthday'], inplace=True)

In [7]:
display(df_users)

,user_id,sex,address,age
0,101391104,Female,"город москва, константинова, дом 30",63
1,101437383,Female,"г. москва, большая серпуховская улица, дом 40,...",64
2,101351338,Female,"город москва, летчика грицевца, дом 10",66
3,101354578,Female,"город москва, лобачевского, дом 76",84
4,101411997,Female,"г. москва, улица вертолётчиков, дом 13",72
...,...,...,...,...
52329,101389206,Female,"город москва, шоссе энтузиастов, дом 48/1",65
52330,101355680,Female,"город москва, аргуновская, дом 16, корпус 1",77
52331,101414122,Male,"город москва, улица генерала белова, дом 41",72
52332,101428464,Female,"г. москва, город зеленоград, корпус 828б",76


### Добавляем улицу проживания

In [8]:
def short_address(address):
    try:
        start = address.find('москва')
        address = address[start+7:]
        address = address[:address.find(',')]
    except:
        address = 'москва'
    address = address.replace('улица','').strip().lower()
    address = address.replace('ул.','').strip().lower()
    return address


df_users['user_address'] = df_users['address'].apply(short_address)

In [9]:
display(df_users)

,user_id,sex,address,age,user_address
0,101391104,Female,"город москва, константинова, дом 30",63,константинова
1,101437383,Female,"г. москва, большая серпуховская улица, дом 40,...",64,большая серпуховская
2,101351338,Female,"город москва, летчика грицевца, дом 10",66,летчика грицевца
3,101354578,Female,"город москва, лобачевского, дом 76",84,лобачевского
4,101411997,Female,"г. москва, улица вертолётчиков, дом 13",72,вертолётчиков
...,...,...,...,...,...
52329,101389206,Female,"город москва, шоссе энтузиастов, дом 48/1",65,шоссе энтузиастов
52330,101355680,Female,"город москва, аргуновская, дом 16, корпус 1",77,аргуновская
52331,101414122,Male,"город москва, улица генерала белова, дом 41",72,генерала белова
52332,101428464,Female,"г. москва, город зеленоград, корпус 828б",76,город зеленоград


### Добавляем координаты

In [10]:
mid = '(55.7498, 37.5371)' # Башня Федерация

# Получаем координаты
df_uniq_streets_coords = pd.read_csv('uniq_streets_coords.csv')
df_uniq_streets_coords = df_uniq_streets_coords[['для координат','Coords']]
df_uniq_streets_coords['для координат']= df_uniq_streets_coords['для координат'].str.lower()
df_uniq_streets_coords['Coords'].fillna(mid, inplace=True)

streets_dict = dict(zip(df_uniq_streets_coords['для координат'], df_uniq_streets_coords['Coords']))

In [11]:
def match_street_coords(street):  
    res = []
    for i in streets_dict.keys():
        if(i.find(street) != -1):
            res.append(streets_dict[i])    
    if len(res) > 0:
        return res[0]
    else:
        return '(55.7498, 37.5371)'
    
    
split_data = df_users['user_address'].apply(match_street_coords).str.strip(')').str.strip('(').str.split(',')
df_users['user_lat'] = split_data.apply(lambda x: x[0]).astype(float)
df_users['user_long'] = split_data.apply(lambda x: x[1]).astype(float)

df_users['user_lat'].fillna(55.7498, inplace=True)
df_users['user_long'].fillna(37.5371, inplace=True)

display(df_users)

,user_id,sex,address,age,user_address,user_lat,user_long
0,101391104,Female,"город москва, константинова, дом 30",63,константинова,56.277942,36.288646
1,101437383,Female,"г. москва, большая серпуховская улица, дом 40,...",64,большая серпуховская,55.729086,37.625461
2,101351338,Female,"город москва, летчика грицевца, дом 10",66,летчика грицевца,55.625444,37.311747
3,101354578,Female,"город москва, лобачевского, дом 76",84,лобачевского,55.663398,37.508764
4,101411997,Female,"г. москва, улица вертолётчиков, дом 13",72,вертолётчиков,55.705414,37.941869
...,...,...,...,...,...,...,...
52329,101389206,Female,"город москва, шоссе энтузиастов, дом 48/1",65,шоссе энтузиастов,55.757569,37.749487
52330,101355680,Female,"город москва, аргуновская, дом 16, корпус 1",77,аргуновская,55.812874,37.619831
52331,101414122,Male,"город москва, улица генерала белова, дом 41",72,генерала белова,55.610416,37.722581
52332,101428464,Female,"г. москва, город зеленоград, корпус 828б",76,город зеленоград,55.996423,37.198534


### Добавляем фичу "Юзер-новичок" в таблицу юзеров

In [12]:
df_users_newbee = pd.read_csv('users_is_newbee_feature_added.csv', encoding='utf-8')
df_users_newbee.rename(columns={'уникальный номер':'user_id',
                                "min(дата занятия)":'first_date',
                                'дельта регистрации и первого занятия дней':'registration_first_attend_delta',
                                'отклоние от медианы дельты':'deviation_from_delta'},
                      inplace=True)
df_users_newbee = df_users_newbee[['user_id','first_date','registration_first_attend_delta','deviation_from_delta','is_newbee']]
display(df_users_newbee)

,user_id,first_date,registration_first_attend_delta,deviation_from_delta,is_newbee
0,101346552,2022-04-01T00:00:00.000+03:00,1491,1.711775,False
1,101346558,2022-04-01T00:00:00.000+03:00,1491,1.711775,False
2,101346562,2022-04-04T00:00:00.000+03:00,1494,1.715219,False
3,101346581,2022-04-14T00:00:00.000+03:00,1500,1.722108,False
4,101346593,2022-04-04T00:00:00.000+03:00,1489,1.709479,False
...,...,...,...,...,...
52329,101451174,NaN,0,0.000000,True
52330,101451200,NaN,0,0.000000,True
52331,101451201,NaN,0,0.000000,True
52332,101451204,NaN,0,0.000000,True


In [13]:
df_users = df_users.merge(df_users_newbee,  how='left', on='user_id')
display(df_users)

,user_id,sex,address,age,user_address,user_lat,user_long,first_date,registration_first_attend_delta,deviation_from_delta,is_newbee
0,101391104,Female,"город москва, константинова, дом 30",63,константинова,56.277942,36.288646,2022-04-01T00:00:00.000+03:00,1129,1.296173,False
1,101437383,Female,"г. москва, большая серпуховская улица, дом 40,...",64,большая серпуховская,55.729086,37.625461,2022-10-04T00:00:00.000+03:00,19,0.021813,True
2,101351338,Female,"город москва, летчика грицевца, дом 10",66,летчика грицевца,55.625444,37.311747,2022-09-06T00:00:00.000+03:00,1631,1.872505,False
3,101354578,Female,"город москва, лобачевского, дом 76",84,лобачевского,55.663398,37.508764,2022-04-01T00:00:00.000+03:00,1470,1.687666,False
4,101411997,Female,"г. москва, улица вертолётчиков, дом 13",72,вертолётчиков,55.705414,37.941869,2022-04-01T00:00:00.000+03:00,835,0.958640,False
...,...,...,...,...,...,...,...,...,...,...,...
52329,101389206,Female,"город москва, шоссе энтузиастов, дом 48/1",65,шоссе энтузиастов,55.757569,37.749487,2023-02-07T00:00:00.000+03:00,1459,1.675037,False
52330,101355680,Female,"город москва, аргуновская, дом 16, корпус 1",77,аргуновская,55.812874,37.619831,2022-07-28T00:00:00.000+03:00,1588,1.823138,False
52331,101414122,Male,"город москва, улица генерала белова, дом 41",72,генерала белова,55.610416,37.722581,2022-06-27T00:00:00.000+03:00,878,1.008007,False
52332,101428464,Female,"г. москва, город зеленоград, корпус 828б",76,город зеленоград,55.996423,37.198534,2022-04-04T00:00:00.000+03:00,13,0.014925,True


### Сохраняем результат

In [14]:
df_users.drop(columns=['address']).to_csv('_users_features.csv')